# Lane Line Detection

**Some OpenCV functions (beyond those introduced in the lesson) that might be useful for this project are:**

`cv2.inRange()` for color selection  
`cv2.fillPoly()` for regions selection  
`cv2.line()` to draw lines on an image given endpoints  
`cv2.addWeighted()` to coadd / overlay two images
`cv2.cvtColor()` to grayscale or change color
`cv2.imwrite()` to output images to file  
`cv2.bitwise_and()` to apply a mask to an image

**Check out the OpenCV documentation to learn about these and discover even more awesome functionality!**

In [14]:
# import dependencies
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
%matplotlib inline
import os
import math
import shutil
import statistics

In [15]:
def region_of_interest(img, vertices):
    """
    Applies an image mask.
    """
    #defining a blank mask to start with
    mask = np.zeros_like(img)   
    
    #defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(img.shape) > 2:
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
        
    #filling pixels inside the polygon defined by "vertices" with the fill color    
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    
    #returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image


def draw_lines(img, lines, color=[255, 0, 0], thickness=2):
    for line in lines:
        for x1,y1,x2,y2 in line:
            cv2.line(img, (x1, y1), (x2, y2), color, thickness)

def hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap):
    """
    Returns an image with hough lines drawn.
    """
    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)
    line_img = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)
    draw_lines(line_img, lines)
    return line_img

# Python 3 has support for cool math symbols.

def weighted_img(img, initial_img, α=0.8, β=1., λ=0.):
    """
    `img` is the output of the hough_lines(), An image with lines drawn on it.
    Should be a blank image (all black) with lines drawn on it.
    
    `initial_img` should be the image before any processing.
    
    The result image is computed as follows:
    
    initial_img * α + img * β + λ
    NOTE: initial_img and img must be the same shape!
    """
    return cv2.addWeighted(initial_img, α, img, β, λ)

## Build a Lane Finding Pipeline



In [16]:
# prepare filesystem for input and output
in_dir_name = "test_images/"
out_dir_name = "result_images/"
if os.path.exists(out_dir_name):
    shutil.rmtree(out_dir_name)
os.mkdir(out_dir_name)
input_image_filenames = os.listdir(in_dir_name)

In [35]:
# set tuning parameters for algortithms
# gaussian blurring param:
kernel_size = 5 # Gaussian blur kernel must be odd number
# canny edge detection params:
low_thresh = 50 # low gradient threshold
hi_thresh = 150 # high gradient threshold
# hough transform params:
rho = 1 # distance resolution of the accumulator in pixels
theta = np.pi/180 # angle resolution of the accumulator in radians
threshold = 2 # votes needed to return line (>threshold).
min_line_len = 50
max_line_gap = 20

for image_filename in input_image_filenames:
    print("processing image: ", image_filename)
    raw_image = cv2.imread(os.path.join(in_dir_name, image_filename))
    gray_image = cv2.cvtColor(raw_image, cv2.COLOR_RGB2GRAY)
    
    # gaussian blur + canny edge detection
    gaussian_image = cv2.GaussianBlur(gray_image, (kernel_size, kernel_size), 0)
    edges_image = cv2.Canny(gaussian_image, low_thresh, hi_thresh)
    # crop
    mask = np.zeros_like(edges_image)   
    ignore_mask_color = 255 
    imshape = raw_image.shape
    vertices = np.array([[(75,imshape[0]), (imshape[1]/2 - 50, imshape[0]/2 + 50), (imshape[1]/2 + 50, imshape[0]/2 + 50), (imshape[1] - 75,imshape[0])]], dtype=np.int32)
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    masked_edges_image = cv2.bitwise_and(edges_image, mask)
    # lines via hough transform
    lines = cv2.HoughLinesP(masked_edges_image, rho, theta, threshold, np.array([]),
                            min_line_len, max_line_gap)
   
    pos_lines = []
    neg_lines = []
    
    for line in lines:
        for x1,y1,x2,y2 in line:
            slope = (y2 - y1) / (x2 - x1)
            if slope > 0:
                pos_lines += [[x1,y1,x2,y2,slope]]
            else:
                neg_lines += [[x1,y1,x2,y2,slope]]
    print("len=",len(pos_lines))
    print(pos_lines)
    print(pos_lines[:][4])
    pos_slope = statistics.median(pos_lines[:][4])
    neg_slope = statistics.median(neg_lines[:][4])
    
    processed_image = masked_edges_image
    cv2.imwrite(os.path.join(out_dir_name, "processed_" + image_filename), processed_image)

processing image:  whiteCarLaneSwitch.jpg
len= 8
[[516, 333, 607, 380, 0.51648351648351654], [823, 505, 873, 532, 0.54000000000000004], [748, 464, 814, 497, 0.5], [744, 468, 840, 524, 0.58333333333333337], [550, 347, 604, 383, 0.66666666666666663], [506, 327, 568, 357, 0.4838709677419355], [809, 507, 863, 539, 0.59259259259259256], [504, 321, 603, 383, 0.6262626262626263]]
[550, 347, 604, 383, 0.66666666666666663]
processing image:  solidYellowCurve2.jpg
len= 7
[[494, 321, 591, 375, 0.55670103092783507], [781, 497, 850, 536, 0.56521739130434778], [500, 323, 591, 373, 0.5494505494505495], [738, 461, 870, 535, 0.56060606060606055], [701, 448, 786, 498, 0.58823529411764708], [500, 320, 592, 373, 0.57608695652173914], [781, 486, 874, 538, 0.55913978494623651]]
[701, 448, 786, 498, 0.58823529411764708]
processing image:  solidYellowCurve.jpg
len= 3
[[491, 320, 582, 374, 0.59340659340659341], [495, 321, 583, 372, 0.57954545454545459], [499, 326, 554, 360, 0.61818181818181817]]


IndexError: list index out of range

## Test on Videos

You know what's cooler than drawing lanes over images? Drawing lanes over video!

We can test our solution on two provided videos:

`solidWhiteRight.mp4`

`solidYellowLeft.mp4`

**Note: if you get an `import error` when you run the next cell, try changing your kernel (select the Kernel menu above --> Change Kernel).  Still have problems?  Try relaunching Jupyter Notebook from the terminal prompt. Also, check out [this forum post](https://carnd-forums.udacity.com/questions/22677062/answers/22677109) for more troubleshooting tips.**

**If you get an error that looks like this:**
```
NeedDownloadError: Need ffmpeg exe. 
You can download it by calling: 
imageio.plugins.ffmpeg.download()
```
**Follow the instructions in the error message and check out [this forum post](https://carnd-forums.udacity.com/display/CAR/questions/26218840/import-videofileclip-error) for more troubleshooting tips across operating systems.**

In [ ]:
# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML

In [ ]:
def process_image(image):
    # NOTE: The output you return should be a color image (3 channel) for processing video below
    # TODO: put your pipeline here,
    # you should return the final output (image where lines are drawn on lanes)

    return result

Let's try the one with the solid white lane on the right first ...

In [ ]:
white_output = 'white.mp4'
clip1 = VideoFileClip("solidWhiteRight.mp4")
white_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
%time white_clip.write_videofile(white_output, audio=False)

Play the video inline, or if you prefer find the video in your filesystem (should be in the same directory) and play it in your video player of choice.

In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(white_output))

## Improve the draw_lines() function

**At this point, if you were successful with making the pipeline and tuning parameters, you probably have the Hough line segments drawn onto the road, but what about identifying the full extent of the lane and marking it clearly as in the example video (P1_example.mp4)?  Think about defining a line to run the full length of the visible lane based on the line segments you identified with the Hough Transform. As mentioned previously, try to average and/or extrapolate the line segments you've detected to map out the full extent of the lane lines. You can see an example of the result you're going for in the video "P1_example.mp4".**

**Go back and modify your draw_lines function accordingly and try re-running your pipeline. The new output should draw a single, solid line over the left lane line and a single, solid line over the right lane line. The lines should start from the bottom of the image and extend out to the top of the region of interest.**

Now for the one with the solid yellow lane on the left. This one's more tricky!

In [ ]:
yellow_output = 'yellow.mp4'
clip2 = VideoFileClip('solidYellowLeft.mp4')
yellow_clip = clip2.fl_image(process_image)
%time yellow_clip.write_videofile(yellow_output, audio=False)

In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(yellow_output))

## Writeup and Submission

If you're satisfied with your video outputs, it's time to make the report writeup in a pdf or markdown file. Once you have this Ipython notebook ready along with the writeup, it's time to submit for review! Here is a [link](https://github.com/udacity/CarND-LaneLines-P1/blob/master/writeup_template.md) to the writeup template file.


## Optional Challenge

Try your lane finding pipeline on the video below.  Does it still work?  Can you figure out a way to make it more robust?  If you're up for the challenge, modify your pipeline so it works with this video and submit it along with the rest of your project!

In [ ]:
challenge_output = 'extra.mp4'
clip2 = VideoFileClip('challenge.mp4')
challenge_clip = clip2.fl_image(process_image)
%time challenge_clip.write_videofile(challenge_output, audio=False)

In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(challenge_output))